In [ ]:
from bs4 import BeautifulSoup
import csv
import pandas as pd
from selenium import webdriver
import winsound

In [ ]:
datelist = pd.date_range("2013-09-30","2019-10-01").tolist()
options = webdriver.ChromeOptions()
options.add_argument("headless")
options.add_argument("--start-maximized");

for d in datelist:
    dateAsString = str(d.date())
    print("https://www.wunderground.com/history/daily/us/va/arlington-county/KDCA/date/"+dateAsString)
      
    try:    
        driver = webdriver.Chrome('E:\\SW\\chromedriver_win32\\chromedriver.exe', options=options)
        driver.get("https://www.wunderground.com/history/daily/us/va/arlington-county/KDCA/date/"+dateAsString)
        soup = BeautifulSoup(driver.page_source,'lxml')

        with open("weatherDataScraped"+dateAsString+".csv","w") as f:
            headerList = []
            for x in soup.find("table",attrs={"class":"tablesaw-sortable"}).find_all("button",attrs={"class":"tablesaw-sortable-btn"}):
                headerList.append(x.text)
            f.write(",".join(headerList))
            f.write("\n")
            for eachTr in soup.find("table",attrs={"class":"tablesaw-sortable"}).find_all("tr"):
                dataList=[]
                for eachSpan in eachTr.find_all("td"):
                    dataList.append(eachSpan.contents[1].text.strip().replace("\n",""))
                if(len(dataList)>0):
                    dataList[0] = dateAsString + " " + dataList[0]
                data = ",".join(dataList)
                if(data!=""):
                    f.write(data)
                    f.write("\n")

        driver.close()
    except:
        print("Failed "+ dateAsString) # improvement possible by keeping track of all failed dates in list
        driver.close()
        
winsound.Beep(2500, 1000)

In [7]:
# aggregating individual CSV files into 1 file
def aggregateDates(combinedDataFrame):

    # convert string into pandas datetime object AND
    # convert per hour data to closest hour based on greater than 30 or less than 30 minutes :: use pandas method
    combinedDataFrame['Time'] = pd.to_datetime(combinedDataFrame['Time'],format="%Y-%m-%d %I:%M %p")
    combinedDataFrame['Time'] = combinedDataFrame['Time'].dt.round("60min")
    
    #convert string into integers
    combinedDataFrame["Temperature"] = pd.to_numeric(combinedDataFrame["Temperature"].str.replace("[^0-9.]",""))
    combinedDataFrame["Dew Point"] = pd.to_numeric(combinedDataFrame["Dew Point"].str.replace("[^0-9.]",""))
    combinedDataFrame["Humidity"] = pd.to_numeric(combinedDataFrame["Humidity"].str.replace("[^0-9.]",""))
    combinedDataFrame["Wind Speed"] = pd.to_numeric(combinedDataFrame["Wind Speed"].str.replace("[^0-9.]",""))
    combinedDataFrame["Wind Gust"] = pd.to_numeric(combinedDataFrame["Wind Gust"].str.replace("[^0-9.]",""))
    combinedDataFrame["Pressure"] = pd.to_numeric(combinedDataFrame["Pressure"].str.replace("[^0-9.]",""))
    combinedDataFrame["Precip."] = pd.to_numeric(combinedDataFrame["Precip."].str.replace("[^0-9.]",""))

    # split apply combine
    # aggregating based on individual columns
    # if multiple hours present then average the numerical values to 1 value, if multiple categorical values present then use least recent weather condition while aggregating
    return combinedDataFrame.groupby("Time",as_index=False).agg({'Condition':'last',
                                                        'Wind':'last',
                                                        "Temperature":"mean",
                                                        "Dew Point":"mean",
                                                        "Humidity":"mean",
                                                        "Wind Speed":"mean",
                                                        "Wind Gust":"mean",
                                                        "Pressure":"mean",
                                                        "Precip.":"mean"}).sort_values("Time")

In [8]:
from os import listdir

filepaths = [f for f in listdir(".") if f.endswith('.csv')] #list comprehension
df = pd.concat(map(pd.read_csv, filepaths))
df.reset_index(inplace=True)

In [9]:
aggregateDates(df).to_csv("aggregatedWeatherData.csv",index=False)